In [1]:
# sim.py

# simulate second half of 2017 NFL season, final standings, playoffs
# by: Jack Barry
# created: 10/31/17
# modified: 9/18/18

import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import calendar
import timeit

pd.set_option('mode.chained_assignment', 'raise')

In [2]:
############################### LOAD DATA FROM PICKLE/CSVs ###############################
# set path to folder
date = datetime.date.isoformat(datetime.date.today())
path = 'pkls/pwr_' + date + '.pkl'

# load team data from pickle
with open(path, 'rb') as handle:
    [pwr] = pickle.load(handle)

# read in schedule
sched = pd.read_csv('sched.csv')
sched.rename(columns={'Unnamed: 2':'Date','Unnamed: 5':'at'}, inplace=True)
####### UPDATE THIS LINE FOR WEEKLY SCHEDULE #######
# sched = sched.loc[sched.Week > 15]
####### UPDATE THIS LINE FOR WEEKLY SCHEDULE #######

# pwr.head()

FileNotFoundError: [Errno 2] No such file or directory: 'pkls/pwr_2019-09-25.pkl'

In [18]:
################################ REGO FUNCTIONS #########################################
#### SIM STUFF ####
# increment team's wins by 1
def teamWin(team, teams):
    teams.loc[team, 'W'] = teams.loc[team, 'W'] + 1
    return teams

# increment team's losses by 1
def teamLoss(team, teams):
    teams.loc[team, 'L'] = teams.loc[team, 'L'] + 1
    return teams

# simulate matchup based on formula
# source: https://www.fangraphs.com/tht/tools-game-and-series-win-probabilities/
def sim_game(home, away, teams):
    # averages, home field advantage
    home_avg = teams.model_pct[home]
    away_avg = teams.model_pct[away]
    adv = 0.55
    # Chance of Team A winning against Team B = 1 / (1+B*(1-A)*(1-H)/A/H/(1-B))
    home_win = 1 / (1+away_avg*(1-home_avg)*(1-adv)/home_avg/adv/(1-away_avg))
    # random sample for comparison
    n = np.random.random_sample()
    # if home win pct is higher than random float, they win
    if(home_win > n):
        teams = teamWin(home, teams)
        teams = teamLoss(away, teams)
    # away wins
    else:
        teams = teamWin(away, teams)
        teams = teamLoss(home, teams)
    return teams

# simulate a week's slate of games 
def sim_week(teams, games):
    for index, game in games.iterrows():
        home = game.loc['HomeTm']
        away = game.loc['VisTm']
        teams = sim_game(home, away, teams)
    return teams

# run an individual season simulation, return dataframe with end of year results
# commented out regular season shit for playoffs
def sim_season(teams, schedule):
#     weeks = range(1,18)
#     for week in weeks:
#         gms = sched.loc[sched.Week == week]
#         teams = sim_week(teams=teams, games=gms)

    # END OF SEASON STUFF
    # win division
    teams = div_winners(teams)

    # wildcard
    teams = wildcard(teams)

    # make playoffs
    teams['make_playoffs'] = teams.win_div + teams.wildcard

    # seeding
    teams = seeding(teams)
    teams.afc_seed.fillna(value=0, inplace=True)
    teams.nfc_seed.fillna(value=0, inplace=True)

    # sim playoffs
    teams = sim_playoffs(teams)
    return teams

In [19]:
################################ END OF SEASON RANKS ################################

# set win_div to 1 for each division winner
def div_winners(teams):
    teams['win_div'] = 0
    div_names = ['AFC_n', 'AFC_s', 'AFC_e', 'AFC_w', 'NFC_n', 'NFC_s', 'NFC_e', 'NFC_w']
    for div in div_names:
        div_df = teams.loc[teams.division==div].sort_values(by=['W', 'MoV', 'SoS'], ascending=False)
        winner = div_df.iloc[0].name
        teams.loc[winner, 'win_div'] = 1
    return teams

# calculate wildcard winners, set wildcard to 1 for the winning teams
def wildcard(teams):
    teams['wildcard'] = 0

    afc = teams.loc[teams.conference == 'AFC']
    nfc = teams.loc[teams.conference == 'NFC']

    afc_wc = afc.loc[afc.win_div == 0]
    nfc_wc = nfc.loc[nfc.win_div == 0]

    afc_wc = afc_wc.sort_values(by=['W', 'MoV', 'SoS'], ascending=False)
    nfc_wc = nfc_wc.sort_values(by=['W', 'MoV', 'SoS'], ascending=False)

    teams.loc[afc_wc.index[0:2], 'wildcard'] = 1 
    teams.loc[nfc_wc.index[0:2], 'wildcard'] = 1 

    return teams

# seed teams 1-6 for playoff bracket
def seeding(teams):
    teams['afc_seed'] = 0
    teams['nfc_seed'] = 0

    afc = teams.loc[teams.conference == 'AFC']
    nfc = teams.loc[teams.conference == 'NFC']
    # separate division winners from wc teams
    afc_div_winners = afc.loc[afc.win_div == 1]
    nfc_div_winners = nfc.loc[nfc.win_div == 1]
    
    afc_wildcard = afc.loc[afc.wildcard == 1]
    nfc_wildcard = nfc.loc[nfc.wildcard == 1]
    
    # rank div winners 1-4 based on criteria, then wc teams 5-6
    afc_div_winners = afc_div_winners.sort_values(by=['W', 'MoV', 'SoS'], ascending=False)
    nfc_div_winners = nfc_div_winners.sort_values(by=['W', 'MoV', 'SoS'], ascending=False)
    afc_div_winners.loc[:,'afc_seed'] = afc_div_winners.afc_seed.rank(method='first')
    nfc_div_winners.loc[:,'nfc_seed'] = nfc_div_winners.nfc_seed.rank(method='first')

    afc_wildcard = afc_wildcard.sort_values(by=['W', 'MoV', 'SoS'], ascending=False)
    nfc_wildcard = nfc_wildcard.sort_values(by=['W', 'MoV', 'SoS'], ascending=False)
    afc_wildcard.loc[:,'afc_seed'] = afc_wildcard.afc_seed.rank(method='first') + 4
    nfc_wildcard.loc[:,'nfc_seed'] = nfc_wildcard.nfc_seed.rank(method='first') + 4
    
    # idk but it works now
    afc_playoffs = pd.concat([afc_div_winners, afc_wildcard])
    nfc_playoffs = pd.concat([nfc_div_winners, nfc_wildcard])
    
    # add wc and div winners back to teams df
    teams.loc[:,'afc_seed'] = afc_playoffs.loc[:,'afc_seed']
    teams.loc[:,'nfc_seed'] = nfc_playoffs.loc[:,'nfc_seed']
    return teams

In [20]:
################################ PLAYOFF FUNCTIONS #########################################

# increment team's wins by 1 in playoffs
def playoff_win(team, teams, rd):
    if (rd == 1):
        teams.loc[team, 'win_rd_1'] = 1
    if (rd == 2):
        teams.loc[team, 'win_rd_2'] = 1
    if (rd == 3):
        teams.loc[team, 'win_rd_3'] = 1
    if (rd == 4):
        teams.loc[team, 'win_rd_4'] = 1
    return teams

# sim playoff game (seeding neutral for superbowl)
def sim_loffs_game(home, away, teams, rd):
    # win percentages for each team
    home_avg = teams.model_pct[home]
    away_avg = teams.model_pct[away]
    # home field advantage changes for super bowl
    if rd == 4: adv = 0.5
    else: adv = 0.55
    # Chance of Team A winning against Team B = 1 / (1+B*(1-A)*(1-H)/A/H/(1-B))
    home_win = 1 / (1+away_avg*(1-home_avg)*(1-adv)/home_avg/adv/(1-away_avg))
    # random sample for comparison
    n = np.random.random_sample()
    # if home win pct is higher than random float, they win
    if(home_win > n):
        teams = playoff_win(home, teams, rd)
    # away wins
    else:
        teams = playoff_win(away, teams, rd)
    return teams

def sim_playoffs(teams):        
    # set up columns for each round
    teams['win_rd_1'] = 0
    teams['win_rd_2'] = 0
    teams['win_rd_3'] = 0
    teams['win_rd_4'] = 0

    # merge afc/nfc seeds
    teams['seed'] = teams.loc[:,'afc_seed'] + teams.loc[:,'nfc_seed']
    # set win_rd_1 for seeds 1 & 2 to 1 (indicating they passed the round)
    teams.loc[teams.seed == 1, 'win_rd_1'] = np.nan
    teams.loc[teams.seed == 2, 'win_rd_1'] = np.nan
    
    # wildcard weekend
    teams = sim_wildcard(teams)  
    # divisional weekend
    teams = sim_divisional(teams)
    # conf champs
    teams = sim_conf(teams)
    # super bowl
    teams = superbowl(teams)
    return teams


##### ROUND BY ROUND #####

# simulate games for wildcard weekend
def sim_wildcard(teams):
    # set home/away teams
    afc = teams[teams.conference == 'AFC']
    afc_home1 = afc.loc[afc.seed == 3].index[0]
    afc_home2 = afc.loc[afc.seed == 4].index[0]
    afc_away1 = afc.loc[afc.seed == 6].index[0]
    afc_away2 = afc.loc[afc.seed == 5].index[0]
    # NFC    
    nfc = teams[teams.conference == 'NFC']
    nfc_home1 = nfc.loc[nfc.seed == 3].index[0]
    nfc_home2 = nfc.loc[nfc.seed == 4].index[0]
    nfc_away1 = nfc.loc[nfc.seed == 6].index[0]
    nfc_away2 = nfc.loc[nfc.seed == 5].index[0]
    
    # simulate wildcard games
    teams = sim_loffs_game(afc_home1, afc_away1, teams, rd=1)
    teams = sim_loffs_game(afc_home2, afc_away2, teams, rd=1)
    teams = sim_loffs_game(nfc_home1, nfc_away1, teams, rd=1)
    teams = sim_loffs_game(nfc_home2, nfc_away2, teams, rd=1)
    
    return teams
    
def sim_divisional(teams):
    # set home/away teams
    afc = teams[teams.conference == 'AFC']
    afc_home1 = afc.loc[afc.seed == 1].index[0]
    afc_home2 = afc.loc[afc.seed == 2].index[0]
    # filter the winners of wc games, set as away teams
    afc = afc[afc.win_rd_1 == 1]
    afc_away1 = afc.seed.idxmax()
    afc_away2 = afc.seed.idxmin()
    # NFC
    nfc = teams[teams.conference == 'NFC']
    nfc_home1 = nfc.loc[nfc.seed == 1].index[0]
    nfc_home2 = nfc.loc[nfc.seed == 2].index[0]
    
    nfc = nfc[nfc.win_rd_1 == 1]
    nfc_away1 = nfc.seed.idxmax()
    nfc_away2 = nfc.seed.idxmin()
    
    # simulate divisional round games
    teams = sim_loffs_game(afc_home1, afc_away1, teams, rd=2)
    teams = sim_loffs_game(afc_home2, afc_away2, teams, rd=2)
    teams = sim_loffs_game(nfc_home1, nfc_away1, teams, rd=2)
    teams = sim_loffs_game(nfc_home2, nfc_away2, teams, rd=2)
    
    return teams

def sim_conf(teams):
    # set home/away teams
    afc = teams[teams.conference == 'AFC']
    afc = afc[afc.win_rd_2 == 1]
    afc_home = afc.seed.idxmin()
    afc_away = afc.seed.idxmax()
    
    # NFC    
    nfc = teams[teams.conference == 'NFC']
    nfc = nfc[nfc.win_rd_2 == 1]
    nfc_home = nfc.seed.idxmin()
    nfc_away = nfc.seed.idxmax()
    
    # simulate conference championships
    teams = sim_loffs_game(afc_home, afc_away, teams, rd=3)
    teams = sim_loffs_game(nfc_home, nfc_away, teams, rd=3)
    
    return teams

def superbowl(teams):
    # set teams (home/away doesn't matter bc HFA is taken out)
    afc = teams[teams.conference == 'AFC']
    afc_champ = afc.loc[afc.win_rd_3 == 1].index[0]
    
    nfc = teams[teams.conference == 'NFC']
    nfc_champ = nfc.loc[nfc.win_rd_3 == 1].index[0]
    
    teams = sim_loffs_game(afc_champ, nfc_champ, teams, 4)
    
    return teams

In [21]:
############################### RUN SIMULATIONS ###############################
start = timeit.default_timer()

# create copy of divs dataframe, that will be populated with results
# results = pd.Panel()
results = pd.read_csv('divs.csv', encoding="ISO-8859-1")
results.set_index('Tm', inplace=True)

# lists to store results
results['W'] = np.empty((len(results), 0)).tolist()
results['L'] = np.empty((len(results), 0)).tolist()
results['win_div'] = np.empty((len(results), 0)).tolist()
results['wildcard'] = np.empty((len(results), 0)).tolist()
results['make_playoffs'] = np.empty((len(results), 0)).tolist()
results['afc_seed'] = np.empty((len(results), 0)).tolist()
results['nfc_seed'] = np.empty((len(results), 0)).tolist()
results['win_rd_1'] = np.empty((len(results), 0)).tolist()
results['win_rd_2'] = np.empty((len(results), 0)).tolist()
results['win_rd_3'] = np.empty((len(results), 0)).tolist()
results['win_rd_4'] = np.empty((len(results), 0)).tolist()

# simulate season 1000 times 
# add end of year W-L totals to each team's dataframe
# results will contain 1000 w-l totals for each team
count = 0
while(count < 5000):
    data = pwr.copy(deep=True)
    # simulate season

    season = sim_season(teams=data, schedule=sched)

    # add season to results
    for index, team in season.iterrows():
        results.loc[index, 'W'].append(team.W)
        results.loc[index, 'L'].append(team.L)
        results.loc[index, 'win_div'].append(team.win_div)
        results.loc[index, 'wildcard'].append(team.wildcard)
        results.loc[index, 'make_playoffs'].append(team.make_playoffs)
        results.loc[index, 'afc_seed'].append(team.afc_seed)
        results.loc[index, 'nfc_seed'].append(team.nfc_seed)
        results.loc[index, 'win_rd_1'].append(team.win_rd_1)
        results.loc[index, 'win_rd_2'].append(team.win_rd_2)
        results.loc[index, 'win_rd_3'].append(team.win_rd_3)
        results.loc[index, 'win_rd_4'].append(team.win_rd_4)

    # iterate counter
#     print(count)
    count = count + 1


stop = timeit.default_timer()
print(stop-start)

306.07394742999895


In [22]:
# cell for testing results
results.head(32)

,conference,division,W,L,win_div,wildcard,make_playoffs,afc_seed,nfc_seed,win_rd_1,win_rd_2,win_rd_3,win_rd_4
Tm,,,,,,,,,,,,,
Philadelphia Eagles,NFC,NFC_e,"[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, ...","[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
Los Angeles Rams,NFC,NFC_w,"[13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13....","[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, ..."
New England Patriots,AFC,AFC_e,"[11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11....","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
Kansas City Chiefs,AFC,AFC_w,"[12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12....","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, ..."
New Orleans Saints,NFC,NFC_s,"[13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13....","[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
Minnesota Vikings,NFC,NFC_n,"[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
Seattle Seahawks,NFC,NFC_w,"[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....","[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [23]:
############################## SAVE DATA ###############################
# set filename
date = datetime.date.isoformat(datetime.date.today())

# dataframes we need to keep
data = [results]

# create pickle object
with open('pkls/sim_results_' +  date + '.pkl', 'wb') as handle:
    pickle.dump(data, handle)